# Usar jupyter para compilar código C/C++

## Que vamos a ver en este cuaderno
1. Creación de fichero fuente y compilación
2. Posibilidad de compilar código por consola

## Ejemplo de uso
Vamos a  ver como podemos usar el google-colaboratory para poder compilar y desarrollar códigos en CUDA:
1. Con la celda mágica **%%writefile** podemos crear un fichero en este caso del tipo C++ que se almacenará en la ruta **/content/hello.cpp**

In [ ]:
%%writefile hello.cpp
#include <iostream>

int main() {
    printf("Hello world my friend!!\n");
    return 0;
}

2. Vamos a compilar como lo haríamos en una consola en este caso con **g++**

In [ ]:
!g++ -o h hello.c

3. Vamos a ejecutar como en la consola, solamente hay que tener en cuenta que para que se ejecute en la consola se ha de añadir el caracter **!**


In [ ]:
!./h

# Compilar código CUDA
* Vamos a hacer lo mismo pero con código CUDA para lo que emplearemos el ejemplo de suma de vectores

In [ ]:
%%writefile vector_add.cu

#include <stdio.h>
#include <malloc.h>

#define N 32

// Compute vector sum C = A+B
__global__ void vecAddkernel(float* A_d, float* B_d, float* C_d, int n)
{
    int i = threadIdx.x + blockDim.x * blockIdx.x;
    if(i<n) C_d[i] = A_d[i] + B_d[i];
}

int main()
{
   float *A_d, *B_d, *C_d;
   int size = N * sizeof(float);
   float *A = (float*)malloc(size);
   float *B = (float*)malloc(size);
   float *C = (float*)malloc(size);

   for(int i=0; i<N; i++){
     A[i] = (float)(i);
     B[i] = 2.0f;
   }

   // Get device memory for A, B, C
   // copy A and B to device memory
   cudaMalloc((void **) &A_d, size);
   cudaMemcpy(A_d, A, size, cudaMemcpyHostToDevice);
   cudaMalloc((void **) &B_d, size);
   cudaMemcpy(B_d, B, size, cudaMemcpyHostToDevice);
   cudaMalloc((void **) &C_d, size);

   int nThread_per_Blocks = 16;
   int nBlocks = N/nThread_per_Blocks;

   // Kernel execution in device
   // (vector add in device)
   vecAddkernel<<<nBlocks, nThread_per_Blocks>>>(A_d, B_d, C_d, N);

   // copy C from device memory
   cudaMemcpy(C, C_d, size, cudaMemcpyDeviceToHost);

   for (int i=0; i<N; i++)
      printf("%f\n", C[i]);

   // free A, B, C
   cudaFree(A_d); cudaFree(B_d); cudaFree (C_d);
}

Compilamos con el compilador **nvcc** y ejecutamos al igual que en el ejemplo anterior

In [ ]:
!nvcc -o vector_add vector_add.cu

In [ ]:
!./vector_add

# Proyecto lab1
* Ahora que hemos visto como poder compilar un ejemplo vamos a visualizar como montar un directorio de Drive donde esté el código a desarrollar y como compilarlo

## Montaje del Drive
* En el apartado **Files** de la ventana a la izquierda se puede montar la unidad *Drive* donde previamente podríamos haber descargado alguna práctica
* Para comprobar el funcionamiento correcto, podemos cambiar al directorio elegido con el comando de python **os.chdir** y la ruta del directorio selecionado

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Es necesario dar permisos de uso del GDrive para poder hacer uso del contenido
* Una vez dados permisos en la "carpeta de la derecha" se puede ir navegando hasta la carpeta deseada **/content/drive/MyDrive/...**

![Image](https://drive.google.com/file/d/1x8Q-1pB0M9-T-TpZ1mMwfVeiMzt4PKA3/view)

* Nos movemos al PATH correspondiente, en mi caso particular a la ruta */content/drive/MyDrive/GPUs/lab1/lane-assist* donde están alojados los ficheros correspondientes al ejemplo del [lane-assist](https://github.com/garsanca/GPUs/blob/main/src/lab1/google-colab/colab_GDrive.png)


In [ ]:
import os
os.chdir("/content/drive/MyDrive/GPUs/lab1/lane-assist")

Listamos los fichero del directorio para asegurarnos que están todos los ficheros correspondientes

In [ ]:
!ls 


Compilamos en este  caso con el **Makefile** [adaptado a Google-Colaboratory](https://github.com/garsanca/GPUs/blob/main/src/lab1/lane-assist/Makefile.colab)

In [ ]:
!make -f Makefile.colab

* Ejecutamos al igual que el ejemplo anterior (usamos la versión cpu: último parámetro a **c**)

In [ ]:
!./image img0.png c

... y visualizamos la imágen de salida que está en el fichero **out.png**

In [ ]:
from IPython.display import Image
Image(filename='out.png') 